In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
from collections import deque
import torch
from torch.nn.attention import flex_attention
from torchtune.modules import attention_utils

score_mod: flex_attention._score_mod_signature | None = None
# score_mods: deque[flex_attention._score_mod_signature | None] = deque()


# We cannot do nested compile, but flex attention only has perf benefits
# when compiled. To insulate it from the compiler, we wrap it with
# compiler.disable so that it can be used regardless of whether the model
# is compiled or not, and flex attention always remains compiled.
@torch.compiler.disable(recursive=False)
def compile_friendly_flex_attention(
    q: torch.Tensor,
    k: torch.Tensor,
    v: torch.Tensor,
    block_mask: flex_attention.BlockMask,
) -> torch.Tensor:
    return attention_utils.flex_attention_compiled(
        q,
        k,
        v,
        score_mod=score_mod,
        block_mask=block_mask,
        kernel_options={
            # "BLOCK_M": 64,
            # "BLOCK_N": 64,
            # "BLOCK_M1": 64,
            # "BLOCK_N1": 64,
            "BLOCK_M2": 64,
            "BLOCK_N2": 64,
        },
    )  # type: ignore


attention_utils.compile_friendly_flex_attention = compile_friendly_flex_attention

In [ ]:
import art
from art.torchtune.config import (
    ModelConfig,
    MetricLoggerConfig,
    OptimizerConfig,
    CheckpointerConfig,
    RecipeConfig,
)
from art.torchtune.recipe import FullFinetuneRecipeDistributed
import asyncio
import glob
import os

process = await asyncio.subprocess.create_subprocess_exec(
    "huggingface-cli",
    "download",
    "Qwen/Qwen3-14B",
    stdout=asyncio.subprocess.PIPE,
    stderr=asyncio.subprocess.PIPE,
)
stdout, _ = await process.communicate()
checkpoint_dir = stdout.decode("utf-8").splitlines()[-1].strip()
safetensor_files = glob.glob(f"{checkpoint_dir}/*.safetensors")
checkpoint_files = sorted(os.path.basename(f) for f in safetensor_files)
print("checkpoint_dir", checkpoint_dir)
print("checkpoint_files", checkpoint_files)

cfg = RecipeConfig(
    model=ModelConfig(_component_="torchtune.models.qwen3.qwen3_14b_instruct"),
    metric_logger=MetricLoggerConfig(
        _component_="torchtune.training.metric_logging.StdoutLogger"
    ),
    optimizer=OptimizerConfig(_component_="torch.optim.AdamW"),
    checkpointer=CheckpointerConfig(
        _component_="torchtune.training.FullModelHFCheckpointer",  # type: ignore
        model_type="QWEN3",
        checkpoint_dir=checkpoint_dir,  # type: ignore
        checkpoint_files=checkpoint_files,  # type: ignore
    ),
    output_dir=os.path.abspath("../.art/temporal-clue/models/002"),
    enable_activation_checkpointing=True,
    enable_activation_offloading=True
)

recipe = FullFinetuneRecipeDistributed(cfg=cfg)  # type: ignore
recipe.setup(cfg=cfg)

In [ ]:
micro_batches, batch = recipe._get_micro_batches(curr_epoch=0)
inputs = micro_batches[0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

In [ ]:
end_index = inputs["tokens"][0].tolist().index(151645, 3190)
end_index

In [ ]:
for key in inputs:
    inputs[key] = inputs[key][:, :end_index]

In [ ]:
print("".join(tokenizer.decode(token_id) for token_id in inputs["tokens"][0]))

In [ ]:
dict(zip(range(-101, -1), [tokenizer.decode(token_id) for token_id in inputs["tokens"][0]][-100:]))

In [ ]:
from torchtune.utils import batch_to_device

batch_to_device(inputs, device=recipe._device) # type: ignore

In [ ]:
import torch
from torch.nn.attention.flex_attention import create_block_mask, BlockMask

def make_block_mask(
    group_ids: torch.Tensor,  # [B, S]  int32/64
    parent_ids: torch.Tensor,  # [B, S]  int32/64
    block_size: int = 256,  # Reduced from 128 to 64 to avoid OOM
) -> BlockMask:
    """
    FlexAttention equivalent of

        causal_mask & (group_ids[q]==group_ids[kv]  |  parent_ids[kv]==group_ids[q])

    * group_ids : id shared by all tokens of the same sampled trajectory
    * parent_ids: id identifying the prompt that produced each token
    """
    B, S = group_ids.shape  # batch, sequence length

    # the closure captures the two id tensors; that's fine for torch.compile
    def mask_mod(b, h, q_idx, kv_idx):
        # causal constraint
        causal = kv_idx <= q_idx

        same_group = group_ids[b, q_idx] == group_ids[b, kv_idx]
        prompt_link = parent_ids[b, q_idx] == group_ids[b, kv_idx]

        return causal & (same_group | prompt_link)

    return create_block_mask(
        mask_mod,
        B=B,
        H=None,
        Q_LEN=S,
        KV_LEN=S,
        BLOCK_SIZE=block_size,
    )

block_mask = make_block_mask(
    group_ids=inputs["group_ids"],
    parent_ids=inputs["parent_ids"],
)

In [ ]:
def calculate_tensor_memory_gb(
    shape: tuple[int, ...],
    dtype: torch.dtype = torch.float32,
    binary_gb: bool = True,
) -> float:
    """
    Calculate the memory usage of a tensor in gigabytes without creating it.

    Args:
        shape: The shape/dimensions of the tensor (e.g., (1024, 1024, 512))
        dtype: The data type of the tensor (e.g., torch.float32, torch.float16)
        binary_gb: If True, use binary GB (1024^3), if False use decimal GB (10^9)

    Returns:
        Memory usage in gigabytes
    """
    # Calculate total number of elements
    total_elements = 1
    for dim in shape:
        total_elements *= dim

    # Get bytes per element based on dtype
    bytes_per_element = torch.tensor([], dtype=dtype).element_size()

    # Calculate total bytes
    total_bytes = total_elements * bytes_per_element

    # Convert to GB
    if binary_gb:
        gb_divisor = 1024**3  # 1 GiB = 1024^3 bytes
    else:
        gb_divisor = 10**9  # 1 GB = 10^9 bytes

    return total_bytes / gb_divisor

calculate_tensor_memory_gb((1, inputs["tokens"].shape[1], inputs["tokens"].shape[1]), dtype=torch.bfloat16)

In [ ]:
attn_bias = torch.zeros((1, inputs["tokens"].shape[1], inputs["tokens"].shape[1]), dtype=torch.bfloat16, device=recipe._device, requires_grad=True)
attn_bias

In [ ]:
score_mod = lambda score, b, h, q_idx, kv_idx: score + attn_bias[b, q_idx, kv_idx]

In [ ]:
# score_mods_list = [score_mod if i % 2 == 0 else None for i in range(len(recipe._model.layers))]
# score_mods = deque(score_mods_list)

In [ ]:
# score_mods = deque([score_mod] + [None] * (len(recipe._model.layers) - 1))
# score_mods = deque([None] * (len(recipe._model.layers) - 1) + [score_mod])

In [ ]:
with recipe.activations_handling_ctx:
    hidden_states = recipe._model(
        tokens=inputs["tokens"],
        mask=block_mask,
        input_pos=inputs["input_pos"],
    )

# del block_mask

In [ ]:
from art.unsloth.train import shift_tensor
from typing import cast

assistant_mask = shift_tensor(inputs["assistant_mask"], False)
hidden_states = hidden_states[assistant_mask]
next_token_ids = shift_tensor(inputs["tokens"], 0)[assistant_mask]
chunk_size = batch.dev_config.get("logprob_calculation_chunk_size", 1024)
logit_sum = torch.tensor(0.0, device=recipe._device)
all_new_logprobs = []
for i in range(0, hidden_states.size(0), chunk_size):
    chunk_end = min(i + chunk_size, hidden_states.size(0))
    # [chunk_size, hidden_size] @ [hidden_size, vocab_size]
    logits = cast(
        torch.Tensor, recipe._model.output(hidden_states[i:chunk_end,:])
    )  # [chunk_size, vocab_size]
    selected_logits = torch.gather(
        logits, dim=-1, index=next_token_ids[i:chunk_end].unsqueeze(-1)
    ).squeeze(
        -1
    )  # [chunk_size]
    logsumexp = torch.logsumexp(logits, dim=-1)  # [chunk_size]
    new_logprobs = selected_logits - logsumexp
    all_new_logprobs.append(new_logprobs)
    # loss += new_logprobs.sum()
    logit_sum += logits.sum()
    # loss += torch.exp(new_logprobs).sum()
    del logits, selected_logits, logsumexp, new_logprobs

new_logprobs = torch.cat(all_new_logprobs, dim=0)
new_logprobs.shape

In [ ]:
# # add L1 loss for attn_bias
# loss += torch.abs(attn_bias).sum() * 10

In [ ]:
# import gc
# import torch

# for _ in range(3):
#     gc.collect()
#     torch.cuda.empty_cache()

In [ ]:
# score_mods = deque([None] * (len(recipe._model.layers) - 1) + [score_mod])
# score_mods = deque(reversed(score_mods_list))

recipe.activations_handling_ctx.repack_tensors = True
result = torch.autograd.grad(logit_sum, attn_bias, retain_graph=True)

In [ ]:
recipe.activations_handling_ctx.is_first_backward_call = True
recipe.activations_handling_ctx.repack_tensors = False

In [ ]:
attn_bias.requires_grad = False

In [ ]:
logit_sum.backward()

In [ ]:
attn_bias.grad is None

In [ ]:
result[0]

In [ ]:
score_mod = None

with recipe.activations_handling_ctx:
    hidden_states = recipe._model(
        tokens=inputs["tokens"],
        mask=block_mask,
        input_pos=inputs["input_pos"],
    )

In [ ]:
hidden_states = hidden_states[assistant_mask]
loss_sum = torch.tensor(0.0, device=recipe._device)
for i in range(0, hidden_states.size(0), chunk_size):
    chunk_end = min(i + chunk_size, hidden_states.size(0))
    # [chunk_size, hidden_size] @ [hidden_size, vocab_size]
    logits = cast(
        torch.Tensor, recipe._model.output(hidden_states[i:chunk_end,:])
    )  # [chunk_size, vocab_size]
    selected_logits = torch.gather(
        logits, dim=-1, index=next_token_ids[i:chunk_end].unsqueeze(-1)
    ).squeeze(
        -1
    )  # [chunk_size]
    logsumexp = torch.logsumexp(logits, dim=-1)  # [chunk_size]
    new_logprobs = selected_logits - logsumexp
    all_new_logprobs.append(new_logprobs)
    loss_sum += new_logprobs.sum()
    # logit_sum += logits.sum()
    # loss += torch.exp(new_logprobs).sum()
    del logits, selected_logits, logsumexp, new_logprobs

In [ ]:
loss_sum *= result[0].sum(dim=1)[assistant_mask].sum()
loss_sum.backward()

In [ ]:
# Analyze attention patterns
import matplotlib.pyplot as plt
import numpy as np

grad_matrix = result[0][0].detach().cpu().float().numpy()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Full heatmap with log scale
ax = axes[0, 0]
im = ax.imshow(np.log10(np.abs(grad_matrix) + 1e-10), cmap='viridis', aspect='auto')
ax.set_title('Log10(|Gradient|) Heatmap')
ax.set_xlabel('Key/Value Position')
ax.set_ylabel('Query Position')
plt.colorbar(im, ax=ax)

# 2. Row-wise sum (gradient strength per query position)
ax = axes[0, 1]
row_sums = np.abs(grad_matrix).sum(axis=1)
ax.plot(row_sums)
ax.set_title('Gradient Strength per Query Position')
ax.set_xlabel('Query Position')
ax.set_ylabel('Sum of Absolute Gradients')
ax.grid(True, alpha=0.3)

# 3. Column-wise sum (gradient strength per key/value position)
ax = axes[1, 0]
col_sums = np.abs(grad_matrix).sum(axis=0)
ax.plot(col_sums)
ax.set_title('Gradient Strength per Key/Value Position')
ax.set_xlabel('Key/Value Position')
ax.set_ylabel('Sum of Absolute Gradients')
ax.grid(True, alpha=0.3)

# 4. Diagonal values (self-attention gradients)
ax = axes[1, 1]
diagonal = np.diag(grad_matrix)
ax.plot(diagonal)
ax.set_title('Diagonal Values (Self-Attention Gradients)')
ax.set_xlabel('Position')
ax.set_ylabel('Gradient Value')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Print pattern analysis
print("Pattern Analysis:")
print(f"- Strongest gradient at position: ({np.unravel_index(np.argmax(np.abs(grad_matrix)), grad_matrix.shape)})")
print(f"- Diagonal mean: {diagonal.mean():.6f}")
print(f"- Off-diagonal mean: {(grad_matrix.sum() - diagonal.sum()) / (grad_matrix.size - len(diagonal)):.6f}")
print(f"- Upper triangular sum: {np.triu(grad_matrix, k=1).sum():.6f}")
print(f"- Lower triangular sum: {np.tril(grad_matrix, k=-1).sum():.6f}")


In [ ]:
inputs["advantages"]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot((grad_matrix.sum(axis=0) + 70_000).cumsum(), color='darkgreen', linewidth=2)
plt.title('Cumulative Sum of Gradient Values by Key/Value Position')
plt.xlabel('Key/Value Position')
plt.ylabel('Cumulative Sum of Gradients')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Plot absolute of sum of gradient values by key/value position
col_sums = grad_matrix.sum(axis=0)
abs_col_sums = col_sums # np.abs(col_sums)

plt.figure(figsize=(12, 6))
plt.plot(abs_col_sums, color='darkgreen', linewidth=2)
plt.title('Absolute of Sum of Gradient Values by Key/Value Position')
plt.xlabel('Key/Value Position')
plt.ylabel('Absolute of Sum of Gradients')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Max absolute sum at position: {np.argmax(abs_col_sums)} (value: {abs_col_sums.max():.6f})")
print(f"Mean absolute sum: {abs_col_sums.mean():.6f}")
print(f"Standard deviation: {abs_col_sums.std():.6f}")



In [ ]:
# Plot gradient strength per query position with rolling average
row_sums = np.abs(grad_matrix).sum(axis=1)

# Calculate rolling average (window size of 50)
window_size = 50
rolling_avg = np.convolve(row_sums, np.ones(window_size)/window_size, mode='valid')

# Create x-axis for rolling average that matches its length
rolling_x = range(window_size//2, window_size//2 + len(rolling_avg))

plt.figure(figsize=(12, 6))
# plt.plot(row_sums, alpha=0.3, label='Raw gradient strength', color='lightblue')
plt.plot(rolling_x, rolling_avg, 
         label=f'Rolling average (window={window_size})', color='darkblue', linewidth=2)
plt.title('Gradient Strength per Query Position (with Rolling Average)')
plt.xlabel('Query Position')
plt.ylabel('Sum of Absolute Gradients')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
result[0][0][-2].to(torch.float32).tolist()

In [ ]:
from illustrate import illustrate

# -44: Professor Plum
# -39: Revolver
# -34: Gazebo
# -18: Ambition
# -13: Betrayal
# -7: Conservatory
# -2: Lounge

print(
    illustrate(
        list(
            zip(
                list(tokenizer.decode(token_id) for token_id in inputs["tokens"][0]),
                result[0][0][-7].to(torch.float32).tolist(),
                # result[0][0].sum(dim=1).squeeze(0).tolist(),
            )
        ),
        gradient="one-dark-simple",
    )
)

In [ ]:
grads = result[0]
grads

In [ ]:
grads[0][-7]

In [ ]:
from illustrate import illustrate

tokens = [tokenizer.decode(token_id) for token_id in inputs["tokens"][0]]
logprobs = torch.exp(new_logprobs).tolist()
print(
    illustrate(
        list(
            zip(
                tokens,
                [0] * (len(tokens) - len(logprobs)) + logprobs,
            )
        ),
        gradient="one-dark-simple",
    )
)

In [ ]:
new_logprobs

In [ ]:
([0] * (len(tokens) - len(logprobs)) + logprobs)[-3]

In [ ]:
result[0].sum(dim=1).squeeze(0).tolist()

In [ ]:
[tokenizer.decode(token_id) for token_id in inputs["tokens"][0]][-100:]

In [ ]:
logit_sum.backward()

In [ ]:
recipe._optimizer.step()
recipe._optimizer.zero_grad()

In [ ]:
hidden_states.shape

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

In [ ]:
recipe._model.layers